In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from workflow.scripts.plotting_tools import global_map, create_facet_plot
from pyclim_noresm.general_util_funcs import global_avg
import numpy as np
import matplotlib as mpl
import pandas as pd

In [3]:
paths = sorted(snakemake.input.paths)
vname = snakemake.wildcards.vName

In [4]:
erfs = {}
erfs['mean'] = {}
erfs['std'] = {}
for path in paths:
    ds = xr.open_dataset(path)
    source = ds.parent_source_id
    ts = ds.mean(dim='year')
    ts = global_avg(ts[vname])
    std = global_avg(ds[vname])
    std = std.std(dim='year')

    erfs['mean'][source] = ts.values
    erfs['std'][source] = std.values
    

In [5]:
df = pd.DataFrame(erfs['mean'],index=['mean'])
df.loc['mean','Multi-model'] = df.loc['mean',:].mean()
df = df.transpose()
vmax=round(abs(df.loc[:,'mean']).max()+0.1,1)

In [6]:
dsets = {}
for path in paths:
    ds = xr.open_dataset(path)
    source_id = ds.parent_source_id
    ds = ds[snakemake.wildcards.vName].mean(dim='year',keep_attrs=True).to_dataset(name=f'{source_id}')
    dsets[source_id]=ds
    
if vname in ['LWDirectEff','LWDirectEff_cs']:
    cvmin=-2
    cvmax=2
else:
    cvmin=-5
    cvmax=5
     

In [7]:
fig,ax = create_facet_plot(len(paths)+1,
                            subplot_kw={'projection':ccrs.PlateCarree()}, 
                            figsize=(14,12), last_axis_plain=True)
for k, i in zip(list(ax.keys())[:-1], dsets.keys()):
    dsets[i][i].plot.contourf(ax=ax[k],levels=np.linspace(cvmin,cvmax,11),cmap='RdYlBu_r', add_colorbar=False)
    ax[k].set_title(i)
    global_map(ax[k])
cax = fig.add_axes([0.94,0.39,0.02,0.47])
cmap = mpl.cm.RdYlBu_r
last = list(ax.keys())[-1]
df.plot.barh(ax=ax[last], legend=False)
ax[last].axvline(color='darkgrey')
ax[last].set_xlim(-(vmax+vmax*0.5),(vmax+vmax*0.5))
ax[last].set_xlabel('$W/m^2$')
ax[last].yaxis.tick_right()
fig.colorbar(mpl.cm.ScalarMappable(mpl.colors.BoundaryNorm(np.linspace(cvmin,cvmax,11),cmap.N),cmap), cax=cax, extend='both', 
             label='$W/m^2$')
plt.savefig(snakemake.output.outpath, bbox_inches='tight', facecolor='white', dpi=144)